<br>
<img src="images/recgov_booking.png" width=80% style="display: inline-block">
<br>
image source: http://www.recreation.gov
<br>
<img src="images/recgov_tou.png" width=80% style="display: inline-block">
<br>
image source: http://www.recreation.gov/marketing.do?goto=/acm/shareData.htm

In [ ]:
def check_recreationgov(facility_url, start_date, stay_length) :
    browser = mechanize.Browser()
    user_agent_str = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"
    browser.addheaders = [('User-agent', user_agent_str)]	
    end_date = "{:%m/%d/%Y}".format(datetime.datetime.strptime(start_date, "%m/%d/%Y") + timedelta(1))

    try:
        browser.open(facility_url)
    except Exception:
        return pd.DataFrame()	

    browser.select_form('unifSearchForm')
    browser.form['arrivalDate'] = start_date
    browser.form['departureDate'] = end_date
    browser.submit()

    date_resp = browser.response()
    availability_html =  date_resp.read()

    soup = BeautifulSoup(availability_html, 'lxml')

    divs = soup.findAll('div', attrs={"class" : 'matchSummary'})
    query_result = divs[0].text
    results = pd.DataFrame()

    for filter_name in soup('div', {"class": re.compile("searchTypeFilter.*")}):
        parts = filter_name.text.split('(')
        entry_name = parts[0]

        name = entry_name.lower().strip().replace(' ', '_')
        name = name.replace("-","")

        # some names have spaces around the '-', some dont
        # i.e. standard - full vs standard-full
        # remove __ to avoid accidentally concatenating standardfull if there arent spaces
        # around the -
        name = name.replace("__", "_")
        results[name] = [parts[1].split(')')[0]]

    results = results.drop('all', axis=1)
    results['last_updated'] = ["{:%m/%d/%Y}".format(datetime.date.today())]
    results['start_date'] = start_date
    results['end_date'] = end_date

    availability_info = query_result.split(' ')
    sites_available = availability_info[0]
    total_sites = availability_info[5]

    results['sites_available'] = [sites_available]
    results['total_sites'] = [total_sites]
    return results